# 📌 Cell 1: List All PDF Files in the Dataset

In [3]:
import os

# Walk through the Kaggle input directory and list only PDF files.
pdf_files = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if filename.lower().endswith(".pdf"):
            file_path = os.path.join(dirname, filename)
            pdf_files.append(file_path)
            #print(file_path)

print(f"\nTotal PDF files found: {len(pdf_files)}")



Total PDF files found: 2484


In [5]:
pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 46.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 54.7 MB/s eta 0:00:00:00:01
Note: you may need to restart the kernel to use updated packages.


# 📌 Cell 2: Improved Text Extraction from PDFs

In [6]:
import pdfplumber

def extract_text_from_pdf(pdf_path):
    """
    Extracts text from a PDF file using pdfplumber.
    
    Parameters:
        pdf_path (str): Path to the PDF file.
    
    Returns:
        text (str): Extracted text.
    """
    text = ""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                page_text = page.extract_text() or ""
                text += page_text + "\n"
    except Exception as e:
        print(f"Error extracting text from {pdf_path}: {e}")
    return text.strip()

# Example: Test the function on the first PDF file (uncomment to test)
# print(extract_text_from_pdf(pdf_files[0])[:500])


# 📌 Cell 3: Load and Process All PDF Resumes

In [7]:
def read_resumes_from_files(file_list):
    """
    Reads all PDF resumes from a list of file paths and extracts text.
    
    Parameters:
        file_list (list): List of PDF file paths.
    
    Returns:
        resumes (list): List of extracted resume texts.
    """
    resumes = []
    for file_path in file_list:
        text = extract_text_from_pdf(file_path)
        if text:  # Only add if text extraction was successful
            resumes.append(text)
    print(f"Total resumes processed: {len(resumes)}")
    return resumes

# Load all resumes from the collected PDF file paths.
all_resumes = read_resumes_from_files(pdf_files)


KeyboardInterrupt: 

# 📌 Cell 4: Preprocess the Extracted Text

In [ ]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Download necessary NLTK data (if not already downloaded)
nltk.download('punkt')
nltk.download('stopwords')

stop_words = set(stopwords.words("english"))

def clean_text(text):
    """
    Cleans resume text by converting to lowercase, removing special characters,
    extra spaces, and stopwords.
    
    Parameters:
        text (str): Original text.
    
    Returns:
        cleaned_text (str): Cleaned text.
    """
    text = text.lower()  # Lowercase
    text = re.sub(r'\s+', ' ', text)  # Remove multiple spaces/newlines
    text = re.sub(r'\W+', ' ', text)  # Remove special characters
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    cleaned_text = " ".join(tokens)
    return cleaned_text

# Preprocess all resumes.
cleaned_resumes = [clean_text(resume) for resume in all_resumes]

print("✅ Text preprocessing complete!")


# 📌 Cell 5: Train a Word2Vec Model on the Resumes

In [ ]:
from gensim.models import Word2Vec

def train_word2vec(resume_texts):
    """
    Trains a Word2Vec model on a list of resume texts.
    
    Parameters:
        resume_texts (list): List of cleaned resume texts.
    
    Returns:
        word2vec_model: Trained Word2Vec model.
    """
    # Tokenize each resume into words.
    tokenized_resumes = [word_tokenize(text) for text in resume_texts]
    # Train Word2Vec model with specified parameters.
    word2vec_model = Word2Vec(sentences=tokenized_resumes, vector_size=100, window=5, min_count=2, workers=4)
    return word2vec_model

# Train the model on the cleaned resumes.
w2v_model = train_word2vec(cleaned_resumes)
print("✅ Word2Vec model trained successfully!")


# 📌 Cell 6: Bias Mitigation – Remove Demographic Indicators

In [ ]:
import spacy

# Load spaCy's small English model for NER (make sure it's available on Kaggle)
nlp = spacy.load("en_core_web_sm")

def remove_demographic_indicators(text):
    """
    Removes demographic indicators (e.g., names, locations) using spaCy's NER.
    
    Parameters:
        text (str): Input text.
    
    Returns:
        cleaned_text (str): Text with demographic entities removed.
    """
    doc = nlp(text)
    # Keep tokens that are not tagged as PERSON or GPE (geopolitical entity)
    tokens = [token.text for token in doc if token.ent_type_ not in ["PERSON", "GPE"]]
    return " ".join(tokens)

# Apply bias mitigation on the cleaned resumes.
debiased_resumes = [remove_demographic_indicators(text) for text in cleaned_resumes]

print("✅ Bias mitigation applied on resume texts!")


# 📌 Cell 7: Enhanced Feedback Generation

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Define a sample job description for ranking (modify as needed)
job_description = "We are seeking a skilled designer with strong experience in graphic design, UI/UX, and creative problem solving."

# Create a TF-IDF vectorizer and fit on the debiased resumes.
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(debiased_resumes)

# Transform the job description into a TF-IDF vector.
job_tfidf = tfidf_vectorizer.transform([job_description])

# Compute cosine similarity between the job description and each resume.
similarities = cosine_similarity(job_tfidf, tfidf_matrix)[0]

# Get ranked indices (highest similarity first).
ranked_indices = similarities.argsort()[::-1]

print("Ranking complete. Top 5 similarity scores:")
for i in range(min(5, len(similarities))):
    print(f"Rank {i+1}: File Index {ranked_indices[i]} with similarity {similarities[ranked_indices[i]]:.4f}")


# 📌 Cell 8: Main Execution – Process, Generate Feedback, and Save Results

In [ ]:
def recruiter_feedback(resume_text):
    """
    Generates feedback for recruiters based on resume quality.
    
    Parameters:
        resume_text (str): Processed resume text.
    
    Returns:
        feedback (str): Feedback message.
    """
    if len(resume_text.split()) < 50:
        return "This resume may lack sufficient details."
    elif "experience" not in resume_text:
        return "Consider looking for resumes with clear experience details."
    else:
        return "Resume appears well-detailed."

def job_seeker_feedback(resume_text):
    """
    Provides feedback for job seekers on how to improve their resumes.
    
    Parameters:
        resume_text (str): Processed resume text.
    
    Returns:
        feedback (str): Feedback message.
    """
    missing_keywords = []
    essential_keywords = ["experience", "skills", "education", "projects"]
    
    for keyword in essential_keywords:
        if keyword not in resume_text:
            missing_keywords.append(keyword)
    
    if missing_keywords:
        return f"Consider adding: {', '.join(missing_keywords)}."
    else:
        return "Your resume appears comprehensive!"

import pandas as pd

results = []
# Use the ranking from TF-IDF; ranked_indices gives the order (best first).
for rank, idx in enumerate(ranked_indices, start=1):
    resume_text = debiased_resumes[idx]
    rec_feedback = recruiter_feedback(resume_text)
    cand_feedback = job_seeker_feedback(resume_text)
    
    results.append({
        "Rank": rank,
        "File Index": idx + 1,  # converting 0-index to 1-index for display
        "Similarity Score": similarities[idx],
        "Recruiter Feedback": rec_feedback,
        "Job Seeker Feedback": cand_feedback,
        "Resume Snippet": resume_text[:500] + "..."
    })

# Convert the results to a DataFrame.
results_df = pd.DataFrame(results)

# Save the results to a CSV file in the working directory.
output_path = "/kaggle/working/resume_feedback_results_with_ranking.csv"
results_df.to_csv(output_path, index=False)
print(f"✅ Results saved to {output_path}")

# Display the first few rows of the results.
results_df.head()


In [ ]:
results_df.head()

# 📌 Cell 9: View a Specific CV (e.g., Top-Ranked Resume)

In [ ]:
from IPython.display import IFrame

# To view the top-ranked resume, use the first element from ranked_indices.
top_resume_index = ranked_indices[0]
top_resume_file = pdf_files[top_resume_index]

print(f"Displaying the top-ranked resume from file: {top_resume_file}")

# Display the PDF inline using IFrame (adjust width and height as desired).
IFrame(top_resume_file, width=800, height=600)


In [ ]:
!lscpu
